In [2]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

from sklearn.metrics import log_loss
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

In [3]:
all_data_path = feature_data_path + 'all_data_all_features.pkl'
all_data = load_pickle(all_data_path)

target = 'is_trade'

In [4]:
features = [
    # 预处理后的基础特征----------------------------------------------
#     'instance_id',
     'item_id',
     'item_brand_id',
     'item_city_id',
     'item_price_level',
     'item_sales_level',
     'item_collected_level',
     'item_pv_level',
     'user_id',
     'user_gender_id',
     'user_age_level',
     'user_occupation_id',
     'user_star_level',
     'context_id',
     'context_timestamp',
     'context_page_id',
     'shop_id',
     'shop_review_num_level',
     'shop_review_positive_rate',
     'shop_star_level',
     'shop_score_service',
     'shop_score_delivery',
     'shop_score_description',
#     'is_trade',
     'day',
     'hour',
#     'minute',
     'category2_label',
     'item_property_list0',
     'item_property_list1',
     'item_property_list2',
     'item_property_list3',
     'item_property_list4',
     'item_property_list5',
     'item_property_list6',
     'item_property_list7',
    #  2_1处理后的特征
    #  生成用户对当天属性的点击量
    #     ['item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level',
    #     'item_collected_level', 'item_pv_level',
    #     'context_page_id',
    #     'shop_id', 'shop_review_num_level', 'shop_star_level',]
     'user_item_id_click_day',
     'user_item_brand_id_click_day',
#     'user_item_city_id_click_day',
     'user_category2_label_click_day',
     'user_item_price_level_click_day',
#     'user_item_sales_level_click_day',
#     'user_item_collected_level_click_day',
#     'user_item_pv_level_click_day',
     'user_context_page_id_click_day',
     'user_shop_id_click_day',
#     'user_shop_review_num_level_click_day',
#     'user_shop_star_level_click_day',
    
    #  2_5处理后的特征
    #  user_id前一天点击某某某的数量
    #  ['item_id', 'item_brand_id', 'shop_id', 'category2_label',]
     'user_id_item_id_day_I',
     'user_id_item_id_day_C',
     'user_id_item_brand_id_day_I',
     'user_id_item_brand_id_day_C',
     'user_id_shop_id_day_I',
     'user_id_shop_id_day_C',
     'user_id_category2_label_day_I',
     'user_id_category2_label_day_C',
     'user_id_item_price_level_day_I',
     'user_id_item_price_level_day_C',
    #  2_5处理后的特征
    #  user_id历史点击某某某的数量
    #  ['item_id', 'item_brand_id', 'shop_id', 'category2_label',]
     'user_id_item_id_history_I',
     'user_id_item_id_history_C',
     'user_id_item_brand_id_history_I',
     'user_id_item_brand_id_history_C',
     'user_id_shop_id_history_I',
     'user_id_shop_id_history_C',
     'user_id_category2_label_history_I',
     'user_id_category2_label_history_C',
     'user_id_item_price_level_history_I',
     'user_id_item_price_level_history_C',
    
    #  2_1处理后的特征
    #  生成用户对当天当小时属性的点击量
    #     ['item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level',
    #     'item_collected_level', 'item_pv_level',
    #     'context_page_id',
    #     'shop_id', 'shop_review_num_level', 'shop_star_level',]
     'user_item_id_click_hour',
     'user_item_brand_id_click_hour',
#     'user_item_city_id_click_hour',
     'user_category2_label_click_hour',
     'user_item_price_level_click_hour',
#     'user_item_sales_level_click_hour',
#     'user_item_collected_level_click_hour',
#     'user_item_pv_level_click_hour',
     'user_context_page_id_click_hour',
     'user_shop_id_click_hour',
#     'user_shop_review_num_level_click_hour',
#     'user_shop_star_level_click_hour',
    #  2_1处理后的特征
    #  生成用户对单一特征点击数据的统计特征
     'user_item_id_click_day_mean',
     'user_item_id_click_day_min',
     'user_item_id_click_day_max',
     'user_item_brand_id_click_day_mean',
     'user_item_brand_id_click_day_min',
     'user_item_brand_id_click_day_max',
     'user_shop_id_click_day_mean',
     'user_shop_id_click_day_min',
     'user_shop_id_click_day_max',
     'user_category2_label_click_day_mean',
     'user_category2_label_click_day_min',
     'user_category2_label_click_day_max',
    
    #  2_2处理后的特征
    #  生成单一特征，日点击量的统计特征    stats_feature = ['user_id', 'item_id', 'item_brand_id', 'shop_id']
     'user_id_click_day_mean',
     'user_id_click_day_max',
     'user_id_click_day_min',
     'item_id_click_day_mean',
     'item_id_click_day_max',
     'item_id_click_day_min',
     'item_brand_id_click_day_mean',
     'item_brand_id_click_day_max',
     'item_brand_id_click_day_min',
     'shop_id_click_day_mean',
     'shop_id_click_day_max',
     'shop_id_click_day_min',
    
    #  2_3处理后的特征
    #  生成用户日点击时间差特征
     'user_click_rank_day',
#     'user_first_click_day',
#     'user_last_click_day',
     'user_click_interval_first_day',
     'user_click_interval_last_day',
#     'user_click_interval_diff_day',
#     'user_click_interval_prob',
     'time_gap_before',
     'time_gap_after',
     'user_click_true_rank_day',
    #  2_3处理后的特征
    #  生成用户全局击时间差特征
     'user_click_interval_mean_hour',
     'time_gap_before_total',
     'time_gap_after_total',
    #  2_3处理后的特征
    #  生成用户对属性全局点击时间差特征
    #   ['item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label',]    
#     'user_item_id_first_click',
#     'user_item_id_last_click',
#     'user_item_id_click_rank',
     'user_item_id_first_click_interval',
     'user_item_id_last_click_interval',
#     'user_item_id_diff_click_interval',
#     'user_item_id_prob_click_interval',
     'item_id_time_gap_before',
     'item_id_time_gap_after',
#     'user_item_id_click_true_rank',
#     'user_item_brand_id_first_click',
#     'user_item_brand_id_last_click',
#     'user_item_brand_id_click_rank',
     'user_item_brand_id_first_click_interval',
     'user_item_brand_id_last_click_interval',
#     'user_item_brand_id_diff_click_interval',
#     'user_item_brand_id_prob_click_interval',
     'item_brand_id_time_gap_before',
     'item_brand_id_time_gap_after',
#     'user_item_brand_id_click_true_rank',
#     'user_shop_id_first_click',
#     'user_shop_id_last_click',
#     'user_shop_id_click_rank',
     'user_shop_id_first_click_interval',
     'user_shop_id_last_click_interval',
#     'user_shop_id_diff_click_interval',
#     'user_shop_id_prob_click_interval',
     'shop_id_time_gap_before',
     'shop_id_time_gap_after',
#     'user_shop_id_click_true_rank',
#      'user_context_page_id_first_click',
#      'user_context_page_id_last_click',
#      'user_context_page_id_click_rank',
#      'user_context_page_id_first_click_interval',
#      'user_context_page_id_last_click_interval',
# #     'user_context_page_id_diff_click_interval',
# #     'user_context_page_id_prob_click_interval',
#      'context_page_id_time_gap_before',
#      'context_page_id_time_gap_after',
#      'user_context_page_id_click_true_rank',
#     'user_category2_label_first_click',
#     'user_category2_label_last_click',
#     'user_category2_label_click_rank',
     'user_category2_label_first_click_interval',
     'user_category2_label_last_click_interval',
#     'user_category2_label_diff_click_interval',
#     'user_category2_label_prob_click_interval',
     'category2_label_time_gap_before',
     'category2_label_time_gap_after',
#     'user_category2_label_click_true_rank',
    #  2_3处理后的特征
    #  生成用户对属性当天点击时间差特征
    #   ['item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label',]    
#     'user_item_id_first_click_day',
#     'user_item_id_last_click_day',
     'user_item_id_click_rank_day',
     'user_item_id_first_click_interval_day',
     'user_item_id_last_click_interval_day',
#     'user_item_id_diff_click_interval_day',
#     'user_item_id_prob_click_interval_day',
     'item_id_time_gap_before_day',
     'item_id_time_gap_after_day',
     'user_item_id_click_true_rank_day',
#     'user_item_brand_id_first_click_day',
#     'user_item_brand_id_last_click_day',
     'user_item_brand_id_click_rank_day',
     'user_item_brand_id_first_click_interval_day',
     'user_item_brand_id_last_click_interval_day',
#     'user_item_brand_id_diff_click_interval_day',
#     'user_item_brand_id_prob_click_interval_day',
     'item_brand_id_time_gap_before_day',
     'item_brand_id_time_gap_after_day',
     'user_item_brand_id_click_true_rank_day',
#     'user_shop_id_first_click_day',
#     'user_shop_id_last_click_day',
     'user_shop_id_click_rank_day',
     'user_shop_id_first_click_interval_day',
     'user_shop_id_last_click_interval_day',
#     'user_shop_id_diff_click_interval_day',
#     'user_shop_id_prob_click_interval_day',
     'shop_id_time_gap_before_day',
     'shop_id_time_gap_after_day',
     'user_shop_id_click_true_rank_day',
#      'user_context_page_id_first_click_day',
#      'user_context_page_id_last_click_day',
#      'user_context_page_id_click_rank_day',
#      'user_context_page_id_first_click_interval_day',
#      'user_context_page_id_last_click_interval_day',
# #     'user_context_page_id_diff_click_interval_day',
# #     'user_context_page_id_prob_click_interval_day',
#      'context_page_id_time_gap_before_day',
#      'context_page_id_time_gap_after_day',
#      'user_context_page_id_click_true_rank_day',
#     'user_category2_label_first_click_day',
#     'user_category2_label_last_click_day',
     'user_category2_label_click_rank_day',
     'user_category2_label_first_click_interval_day',
     'user_category2_label_last_click_interval_day',
#     'user_category2_label_diff_click_interval_day',
#     'user_category2_label_prob_click_interval_day',
     'category2_label_time_gap_before_day',
     'category2_label_time_gap_after_day',
     'user_category2_label_click_true_rank_day',
    
    #  2_4处理后的特征
     'property_sim',
     'category_predict_rank',
     'category_3',
    
    #  2_5处理后的特征
    #  生成单特征历史点击率，要去除点击次数和点击时间
    #     ['user_id', 'category_predict_rank', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
    #     'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level',
    #     'item_collected_level', 'item_pv_level',
    #     'context_page_id',
    #     'shop_id', 'shop_review_num_level', 'shop_star_level',]    
     'user_id_smooth_CTR',
     'category_predict_rank_smooth_CTR',
     'user_occupation_id_smooth_CTR',
     'user_age_level_smooth_CTR',
     'user_gender_id_smooth_CTR',
     'user_star_level_smooth_CTR',
     'item_id_smooth_CTR',
     'item_brand_id_smooth_CTR',
     'item_city_id_smooth_CTR',
     'category2_label_smooth_CTR',
     'item_price_level_smooth_CTR',
     'item_sales_level_smooth_CTR',
     'item_collected_level_smooth_CTR',
     'item_pv_level_smooth_CTR',
     'context_page_id_smooth_CTR',
     'shop_id_smooth_CTR',
     'shop_review_num_level_smooth_CTR',
     'shop_star_level_smooth_CTR',
    #  2_5处理后的特征
    #  生成单特征前一天点击率，前一天的点击次数，前一天的购买次数， 去除ctr
    #     ['user_id', 'category_predict_rank', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
    #     'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level',
    #     'item_collected_level', 'item_pv_level',
    #     'context_page_id',
    #     'shop_id', 'shop_review_num_level', 'shop_star_level',]
     'user_id_day_I',
     'user_id_day_C',
#     'category_predict_rank_day_I',
#     'category_predict_rank_day_C',
#     'user_occupation_id_day_I',
#     'user_occupation_id_day_C',
#     'user_age_level_day_I',
#     'user_age_level_day_C',
#     'user_gender_id_day_I',
#     'user_gender_id_day_C',
#     'user_star_level_day_I',
#     'user_star_level_day_C',
     'item_id_day_I',
     'item_id_day_C',
     'item_brand_id_day_I',
     'item_brand_id_day_C',
#     'item_city_id_day_I',
#     'item_city_id_day_C',
     'category2_label_day_I',
     'category2_label_day_C',
#     'item_price_level_day_I',
#     'item_price_level_day_C',
#     'item_sales_level_day_I',
#     'item_sales_level_day_C',
#     'item_collected_level_day_I',
#     'item_collected_level_day_C',
#     'item_pv_level_day_I',
#     'item_pv_level_day_C',
#     'context_page_id_day_I',
#     'context_page_id_day_C',
     'shop_id_day_I',
     'shop_id_day_C',
#     'shop_review_num_level_day_I',
#     'shop_review_num_level_day_C',
#     'shop_star_level_day_I',
#     'shop_star_level_day_C',

    #  2_5处理后的特征
    #  生成历史交叉点击率，前一天的点击次数，前一天的购买次数，只保留ctr
    #    ['user_gender_id', 'user_age_level', 'user_occupation_id']
    #    ['item_id', 'item_brand_id', 'shop_id']    

     'user_gender_id_item_id_smooth_CTR',
     'user_gender_id_item_brand_id_smooth_CTR',
     'user_gender_id_shop_id_smooth_CTR',
     'user_gender_id_item_price_level_smooth_CTR',
     'user_age_level_item_id_smooth_CTR',
     'user_age_level_item_brand_id_smooth_CTR',
     'user_age_level_shop_id_smooth_CTR',
     'user_age_level_item_price_level_smooth_CTR',
     'user_occupation_id_item_id_smooth_CTR',
     'user_occupation_id_item_brand_id_smooth_CTR',
     'user_occupation_id_shop_id_smooth_CTR',
     'user_occupation_id_item_price_level_smooth_CTR',
#     'user_star_level_item_id_smooth_CTR',
#     'user_star_level_item_brand_id_smooth_CTR',
#     'user_star_level_shop_id_smooth_CTR',
     'user_star_level_item_price_level_smooth_CTR',

    
    #  2_6处理后的特征
    #  分别groupby['shop_id', 'item_id', 'item_brand_id', 'item_price_level']
    #    计算item在['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']几个属性下的点击量
     'shop_id_user_gender_id_click_rate',
     'shop_id_user_age_level_click_rate',
     'shop_id_user_occupation_id_click_rate',
#     'shop_id_user_star_level_click_rate',
     'item_id_user_gender_id_click_rate',
     'item_id_user_age_level_click_rate',
     'item_id_user_occupation_id_click_rate',
#     'item_id_user_star_level_click_rate',
     'item_brand_id_user_gender_id_click_rate',
     'item_brand_id_user_age_level_click_rate',
     'item_brand_id_user_occupation_id_click_rate',
#     'item_brand_id_user_star_level_click_rate',
     'item_price_level_user_gender_id_click_rate',
     'item_price_level_user_age_level_click_rate',
     'item_price_level_user_occupation_id_click_rate',
     'item_price_level_user_star_level_click_rate',
    
    #  2_7处理后的特征
    #  计算每天的点击量
    #     ['user_id', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
    #      'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level',
    #      'item_collected_level', 'item_pv_level',
    #      'context_page_id',
    #      'shop_id', 'shop_review_num_level', 'shop_star_level',]
     'user_id_click_day',
     'user_occupation_id_click_day',
     'user_age_level_click_day',
     'user_gender_id_click_day',
     'user_star_level_click_day',
     'item_id_click_day',
     'item_brand_id_click_day',
#     'item_city_id_click_day',
     'category2_label_click_day',
     'item_price_level_click_day',
     'item_sales_level_click_day',
     'item_collected_level_click_day',
#     'item_pv_level_click_day',
     'context_page_id_click_day',
     'shop_id_click_day',
#     'shop_review_num_level_click_day',
#     'shop_star_level_click_day',
    #  2_7处理后的特征
    #  计算每天每小时的点击量
    #     ['user_id', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
    #      'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level',
    #      'item_collected_level', 'item_pv_level',
    #      'context_page_id',
    #      'shop_id', 'shop_review_num_level', 'shop_star_level',]
     'user_id_click_hour_x',
     'user_occupation_id_click_hour_x',
     'user_age_level_click_hour_x',
     'user_gender_id_click_hour_x',
     'user_star_level_click_hour_x',
     'item_id_click_hour_x',
     'item_brand_id_click_hour_x',
#     'item_city_id_click_hour_x',
     'category2_label_click_hour_x',
     'item_price_level_click_hour_x',
     'item_sales_level_click_hour_x',
     'item_collected_level_click_hour_x',
#     'item_pv_level_click_hour_x',
#     'context_page_id_click_hour_x',
     'shop_id_click_hour_x',
#     'shop_review_num_level_click_hour_x',
#     'shop_star_level_click_hour_x',
    #  2_7处理后的特征
    #  计算每小时的点击量
    #     ['user_id', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
    #      'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level',
    #      'item_collected_level', 'item_pv_level',
    #      'context_page_id',
    #      'shop_id', 'shop_review_num_level', 'shop_star_level',]
     'user_id_click_hour_y',
     'user_occupation_id_click_hour_y',
     'user_age_level_click_hour_y',
     'user_gender_id_click_hour_y',
#     'user_star_level_click_hour_y',
     'item_id_click_hour_y',
     'item_brand_id_click_hour_y',
#     'item_city_id_click_hour_y',
     'category2_label_click_hour_y',
#     'item_price_level_click_hour_y',
#     'item_sales_level_click_hour_y',
#     'item_collected_level_click_hour_y',
#     'item_pv_level_click_hour_y',
#     'context_page_id_click_hour_y',
     'shop_id_click_hour_y',
#     'shop_review_num_level_click_hour_y',
#     'shop_star_level_click_hour_y'
]
len(features)

228

In [7]:
def CustomCV(data, k=1):
    train_days = 6 - k
    for i in range(k):
        train_start_day = 19 + i
        train_end_day = 19 + i + train_days - 1
        test_day = 19 + i + train_days
        fold_index_train = data[(data.day >= train_start_day) & (data.day <= train_end_day)].index
        fold_index_test = data[data.day == test_day].index
        yield fold_index_train, fold_index_test
    

In [29]:
def CustomCV_19_21_22(data, k=1):
    fold_index_train = data[(data.day >= 19) & (data.day <= 21)].index
    fold_index_test = data[data.day == 22].index
    yield fold_index_train, fold_index_test

In [30]:
def CustomCV_19_22_23(data, k=1):
    fold_index_train = data[(data.day >= 19) & (data.day <= 22)].index
    fold_index_test = data[data.day == 23].index
    yield fold_index_train, fold_index_test

In [31]:
def CustomCV_20_23_24(data, k=1):
    fold_index_train = data[(data.day >= 20) & (data.day <= 23)].index
    fold_index_test = data[data.day == 24].index
    yield fold_index_train, fold_index_test

In [37]:
def CustomCV_19_23_24(data, k=1):
    fold_index_train = data[(data.day >= 19) & (data.day <= 23)].index
    fold_index_test = data[data.day == 24].index
    yield fold_index_train, fold_index_test

In [40]:
if __name__ == '__main__':


    eval_data = all_data[all_data.day == 23]
    eval_set = [(eval_data[features], eval_data[target])]

    lgb_clf = lgb.LGBMClassifier(objective='binary', device='gpu',  n_jobs=-1, silent=False)

    # 参数的组合
    lgb_param_grad = {'n_estimators': (2000, ),
                      'learning_rate': (0.02, ),

                      'max_depth': (4,),
                      'num_leaves': (20,),
                      'min_child_samples': (100, 115, 130, ),
#                       'min_child_weight': (1e-3, ),

#                       'colsample_bytree': (1.0, ),
                      'subsample': (0.7, ),
#                       'subsample_freq': (1, ),
#                       'min_split_gain': (0, ),
                      
#                       'reg_alpha': (0,),
                      'reg_lambda': (10,),
                      
                      'max_bin': (63, ),
                      
                      'gpu_use_dp': (False, ),
                      }

    clf = GridSearchCV(lgb_clf, param_grid=lgb_param_grad, scoring='neg_log_loss',
                       cv=CustomCV(all_data, k=1), n_jobs=-1, verbose=1, refit=False, return_train_score=True)

    clf.fit(all_data[features], all_data[target],
            feature_name=features,
#             categorical_feature=nominal_feats,
            early_stopping_rounds=100, eval_set=eval_set, eval_names='test_day_24', verbose=100
           )

    print('=====')
    print("Best parameters set found on development set:")
    print(clf.best_params_)

    print('=====')
    print("Best parameters set found on development set:")
    print(clf.best_score_)


Fitting 1 folds for each of 3 candidates, totalling 3 fits
Training until validation scores don't improve for 100 rounds.
Training until validation scores don't improve for 100 rounds.
[100]	test_day_24's binary_logloss: 0.126799
Training until validation scores don't improve for 100 rounds.
[200]	test_day_24's binary_logloss: 0.0842621
[100]	test_day_24's binary_logloss: 0.126804
[100]	test_day_24's binary_logloss: 0.126805
[300]	test_day_24's binary_logloss: 0.0805307
[200]	test_day_24's binary_logloss: 0.0842734
[200]	test_day_24's binary_logloss: 0.0842698
[400]	test_day_24's binary_logloss: 0.0797914
[500]	test_day_24's binary_logloss: 0.0795134
[300]	test_day_24's binary_logloss: 0.0805346
[300]	test_day_24's binary_logloss: 0.0805322
[600]	test_day_24's binary_logloss: 0.0793986
[400]	test_day_24's binary_logloss: 0.0798099
[400]	test_day_24's binary_logloss: 0.0798065
[700]	test_day_24's binary_logloss: 0.0793373
[500]	test_day_24's binary_logloss: 0.0795239
[800]	test_day_24's

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.0min finished


=====
Best parameters set found on development set:
{'gpu_use_dp': False, 'learning_rate': 0.02, 'max_bin': 63, 'max_depth': 4, 'min_child_samples': 100, 'n_estimators': 2000, 'num_leaves': 20, 'reg_lambda': 10, 'subsample': 0.7}
=====
Best parameters set found on development set:
-0.0792922701161


In [35]:
pd.DataFrame(data=clf.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_gpu_use_dp,param_learning_rate,param_max_bin,param_max_depth,param_min_child_samples,param_n_estimators,...,param_reg_lambda,param_subsample,params,rank_test_score,split0_test_score,split0_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,107.185586,1.559737,-0.084575,-0.081227,False,0.02,63,4,100,2000,...,10,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",3,-0.084575,-0.081227,0.0,0.0,0.0,0.0
1,14.380348,1.313157,-0.693147,-0.693147,False,0.02,63,4,100,2000,...,15,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",4,-0.693147,-0.693147,0.0,0.0,0.0,0.0
2,19.321992,1.370973,-0.693147,-0.693147,False,0.02,63,4,100,2000,...,20,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",4,-0.693147,-0.693147,0.0,0.0,0.0,0.0
3,99.437652,1.218781,-0.084572,-0.081227,False,0.02,63,4,100,2000,...,10,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",2,-0.084572,-0.081227,0.0,0.0,0.0,0.0
4,20.971287,1.055903,-0.693147,-0.693147,False,0.02,63,4,100,2000,...,15,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",4,-0.693147,-0.693147,0.0,0.0,0.0,0.0
5,21.030164,1.155212,-0.693147,-0.693147,False,0.02,63,4,100,2000,...,20,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",4,-0.693147,-0.693147,0.0,0.0,0.0,0.0
6,88.831379,0.742936,-0.084570,-0.081227,False,0.02,63,4,100,2000,...,10,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",1,-0.084570,-0.081227,0.0,0.0,0.0,0.0
7,25.464741,2.335806,-0.693147,-0.693147,False,0.02,63,4,100,2000,...,15,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",4,-0.693147,-0.693147,0.0,0.0,0.0,0.0
8,23.145646,1.028846,-0.693147,-0.693147,False,0.02,63,4,100,2000,...,20,0.7,"{'gpu_use_dp': False, 'learning_rate': 0.02, '...",4,-0.693147,-0.693147,0.0,0.0,0.0,0.0
